Main
Michael de Jong

In [4]:
# imports
import os
import json
from elasticsearch import Elasticsearch
import numpy as np
import pandas as pd
import nbconvert
import nbformat
import zipfile
import csv
import time

In [51]:
# load local python file(s)
%run evaluationmetrics.py

In [52]:
# 1 = relevant, 0 = not relevant
l = [1,1,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,1]
c = 8
print(precision(l))
print(recall(l,c))
print(f1(l,c))
print(avg_precision(l,c))

0.3
0.75
0.4285714285714285
0.262359022556391


In [2]:
# connect to local elastic search host
HOST = 'http://localhost:9200/'
es = Elasticsearch(hosts=[HOST])

# global vars
INDEX="notebookindex"
TYPE= "notebook"
COLUMNS = ['nb_id', 'html_url', 'name', 'language', 'markdown', 'comments', 'code']

In [3]:
# load the csv with all notebook information
df_nb = pd.read_csv('notebooks.csv')
df_nb = df_nb.drop(['max_filesize','min_filesize', 'path', 'query_page', 'repo_id'], axis=1)
#print('%s notebooks' % df_nb.shape[0])
df_nb.head()

,nb_id,html_url,name
0,0,https://github.com/dalequark/emotivExperiment/...,EmotivDataAnalysis.ipynb
1,1,https://github.com/kevcisme/madelon_redux/blob...,Part_IV_Project_3-checkpoint_BASE_63907.ipynb
2,2,https://github.com/HaraldoFilho/DLND-Projects/...,_.ipynb
3,3,https://github.com/mhjensen/CPMLS/blob/4a5b37e...,csexmas2015.ipynb
4,4,https://github.com/freqn/atom_configuration/bl...,jupyter.ipynb


In [5]:
folder = 'bb2733859v_2_1'
# filenames = os.listdir(folder)
# print(len(filenames))
def get_ids(path):
    filenames = os.listdir(path)
    ids = []
    for i in range(len(filenames)):
        if filenames[i].startswith('nb_') and filenames[i].endswith('.ipynb'):
            current_id = int(filenames[i][3:-6])
            ids.append(current_id)
        else:
            print('miss')
    return ids

def get_ids_zip(path):
    path = path + '.zip'
    nbzip = zipfile.ZipFile(path, 'r')
    filenames = nbzip.namelist()
    ids = []
    for i in range(len(filenames)):
        if filenames[i].startswith('nb_') and filenames[i].endswith('.ipynb'):
            current_id = int(filenames[i][3:-6])
            ids.append(current_id)
        else:
            print('miss')
    return ids, nbzip

#ids = get_ids(folder)
ids, nbzip = get_ids_zip(folder)
# print(len(ids)) # check same length
# row = df_nb.loc[df_nb['nb_id'] == 0]

In [20]:
# extract markdown, language and comments
# path = "./sample_data/data/notebooks/"
# notebook_loc = path + 'nb_1222.ipynb'
def get_text(path):
    markdown = []
    comments = []
    code = []
    with open(path) as fp:
        try:
            data = nbformat.read(fp, nbformat.NO_CONVERT)
            #data = json.load(fp)
        except:
            return None, None, None
    if 'cells' in data:
        cells = data['cells']
        if 'metadata' in data:
            if 'kernelspec' in data['metadata']:
                if 'language' in data['metadata']['kernelspec']:
                    language = data['metadata']['kernelspec']['language']
                else:
                    language = None
            else:
                language = None
        else:
            language = None
        md_cells = [c for c in cells if c['cell_type'] == 'markdown']
        code_cells = [c for c in cells if c['cell_type'] == 'code']
        for cell in md_cells:
            markdown.append(cell['source'])

        # find comments '# ' for R and Python
        for cell in code_cells:
            source = cell['source']
            code.append(source)
            string = ''
            if source != None:
                for item in source:
                    string = string + str(item)
                if '# ' in string:
                    comments.append(string)
        return language, markdown, comments, code
    else:
        return None, None, None, None
    
# zip version
def get_text_zip(nb_id, nbzip):
    markdown = []
    comments = []
    code = []
    # raw = nbzip.read('nb_' + str(nb_id) + '.ipynb')
    with nbzip.open('nb_' + str(nb_id) + '.ipynb') as raw:
        try:
            data = nbformat.read(raw, nbformat.NO_CONVERT)
            #data = json.load(fp)
        except:
            return None, None, None, None
    if 'cells' in data:
        cells = data['cells']
        if 'metadata' in data:
            if 'kernelspec' in data['metadata']:
                if 'language' in data['metadata']['kernelspec']:
                    language = data['metadata']['kernelspec']['language']
                else:
                    language = None
            else:
                language = None
        else:
            language = None
        md_cells = [c for c in cells if c['cell_type'] == 'markdown']
        code_cells = [c for c in cells if c['cell_type'] == 'code']
        for cell in md_cells:
            markdown.append(cell['source'])

        # find comments '# ' for R and Python
        for cell in code_cells:
            source = cell['source']
            code.append(source)
            string = ''
            if source != None:
                for item in source:
                    string = string + str(item)
                if '# ' in string:
                    comments.append(string)
        return language, markdown, comments, code
    else:
        return None, None, None, None

# l,m,c = get_text(notebook_loc)
#print(c)
# print(get_text('bb2733859v_2_1/nb_0.ipynb'))
print(get_text_zip(100002, nbzip))

('python', ['### Therefore Training accuracy of 93.50% and Validation accuracy of 96.6% achieved in 200 epochs of a neural network with 1 hidden layer comprising of 4 neurons.'], ["# Keras model\n\nmodel = Sequential()\n\nfc1 = Dense(4, input_shape=(2,))           #This is the first hidden layer, therefore sigmoid function as activation and 4 neurons\nmodel.add(fc1)\nmodel.add(Activation('sigmoid'))\n\nfc2 = Dense(3)                             #Output layer, don't need to mention input- it will take that from previous layer automatically\nmodel.add(fc2)\nmodel.add(Activation('softmax'))\n\nmodel.summary()\nmodel.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])"], ['import numpy as np\nfrom matplotlib import pyplot as plt\n%matplotlib inline\n\n#----------------keras imports-------------------#\nimport keras\nfrom keras.layers import Input, Dense, Activation\nfrom keras.models import Sequential\nfrom keras.utils import np_utils', 'mean_01 = np.array([3.0,

In [28]:
# WITHOUT ZIP
# append all extra columns to dataframe and export to csv
begintime = time.time()
final_df = pd.DataFrame(columns = COLUMNS)
for i in range(len(ids)):
    path = folder + '/nb_' + str(ids[i]) + '.ipynb'
    language, markdown, comments = get_text(path)
    if language != None or markdown != None or comments != None:
        row = df_nb.loc[df_nb['nb_id'] == ids[i]]
        final_df = final_df.append({COLUMNS[0]:row['nb_id'].values[0],
                                    COLUMNS[1]:row['html_url'].values[0],
                                    COLUMNS[2]:row['name'].values[0],
                                    COLUMNS[3]:language,
                                    COLUMNS[4]:markdown,
                                    COLUMNS[5]:comments,
                                    COLUMNS[6]:code},
                                   ignore_index=True)
# save the dataframe to csv
midtime = time.time()
print(midtime - starttime)
final_df.to_csv('df_bb2733859v_2_1.csv')
endtime = time.time()
print(endtime - starttime)
final_df.head()

O:\Anaconda3\lib\site-packages\nbformat\validator.py:251: UserWarning: No schema for validating v2 notebooks
  warnings.warn("No schema for validating v%s notebooks" % version, UserWarning)
O:\Anaconda3\lib\site-packages\nbformat\validator.py:251: UserWarning: No schema for validating v1 notebooks
  warnings.warn("No schema for validating v%s notebooks" % version, UserWarning)


,nb_id,html_url,name,language,markdown,comments
0,1000,https://github.com/crackcoder1291/SampleCard/b...,MeanMedianExercise.ipynb,python,"[# Exercise: Mean & Median Customer Spend, Her...",[]
1,100001,https://github.com/Yanhao29/UW_MachineLearning...,week-5-lasso-assignment-1-blank.ipynb,python,[# Regression Week 5: Feature Selection and LA...,"[from math import log, sqrt\nsales['sqft_livin..."
2,100002,https://github.com/navjot12/Mini_ML_Projects/b...,Neural Network with 3 layers-checkpoint.ipynb,python,[### Therefore Training accuracy of 93.50% and...,[# Keras model\n\nmodel = Sequential()\n\nfc1 ...
3,100003,https://github.com/navjot12/Mini_ML_Projects/b...,Neural Network with 3 layers.ipynb,python,[### Therefore Training accuracy of 93.50% and...,[# Keras model\n\nmodel = Sequential()\n\nfc1 ...
4,100004,https://github.com/yama1968/Spikes/blob/46e450...,my_bayesian_survival_large-checkpoint.ipynb,python,[],[# new model\n\ncensor = (lifetime + birth) > ...


In [21]:
# append all extra columns to dataframe and export to csv, zip 
starttime = time.time()
final_df = pd.DataFrame(columns = COLUMNS)
for i in range(len(ids)):
    language, markdown, comments, code = get_text_zip(ids[i], nbzip)
    if language != None or markdown != None or comments != None or code != None:
        row = df_nb.loc[df_nb['nb_id'] == ids[i]]
        final_df = final_df.append({COLUMNS[0]:row['nb_id'].values[0],
                                    COLUMNS[1]:row['html_url'].values[0],
                                    COLUMNS[2]:row['name'].values[0],
                                    COLUMNS[3]:language,
                                    COLUMNS[4]:markdown,
                                    COLUMNS[5]:comments,
                                    COLUMNS[6]:code},
                                   ignore_index=True)
# save the dataframe to pickle
midtime = time.time()
print(midtime - starttime)
final_df.to_pickle('df_bb2733859v_2_1.pkl')
endtime = time.time()
print(endtime - starttime)
final_df.head()

O:\Anaconda3\lib\site-packages\nbformat\validator.py:251: UserWarning: No schema for validating v2 notebooks
  warnings.warn("No schema for validating v%s notebooks" % version, UserWarning)
O:\Anaconda3\lib\site-packages\nbformat\validator.py:251: UserWarning: No schema for validating v1 notebooks
  warnings.warn("No schema for validating v%s notebooks" % version, UserWarning)


16182.617602348328
16209.22017788887


,nb_id,html_url,name,language,markdown,comments,code
0,120951,https://github.com/tuckercs/DSI-Projects/blob/...,Fizz Buzz.ipynb,python,[],[],"[import pandas as pd, def fizzbuzz(n):\n if..."
1,23352,https://github.com/Isabella-Dineo/beam_py/blob...,patchbeam_main.ipynb,python,[],"[def correct(x):\n """"""Function to correct f...",[import numpy as np\nimport scipy as sp\nfrom ...
2,73638,https://github.com/wanglongjuan/15-ODE-homewor...,2015750221张杰-checkpoint.ipynb,,"[数学与应用数学班, 张杰 2015750221, **P88——3**, **求初值问题$...",[],[]
3,179265,https://github.com/tyui592/nn_study/blob/f3c37...,hand_write_function_learning.ipynb,python,"[# Artificial Neural Network, ## Object : hand...",[# read hand write function image\nimg = color...,"[from skimage import io, color\nimport matplot..."
4,54493,https://github.com/suhagba/Naive_Bayes/blob/e6...,Tutorial5.ipynb,python,[**Name:** Byaravalli Arun Suhag\n\n**EID:** 5...,[%matplotlib inline\nimport IPython.core.displ...,[%matplotlib inline\nimport IPython.core.displ...


In [6]:
es_df = pd.read_csv('df_bb2733859v_2_1.csv')

bulk_data = []

for index, row in es_df.iterrows():
    data_dict = {}
    for i in range(len(row)):
        data_dict[es_df.columns[i]] = row[i]
    op_dict = {
        "index": {
            "_index": INDEX,
            "_type": TYPE,
            "_id": es_df['nb_id']
        }
    }
    bulk_data.append(op_dict)
    bulk_data.append(data_dict)

In [23]:
# Initialise the elastic search index
init_index = {
    "settings" : {
        "number_of_shards": 5,
        "number_of_replicas": 1,
        "analysis": {
            "analyzer": {
                "text_analyzer": {
                    "type": "standard",
                    "stopwords": "_english_"
                }
            }
        }
    },
    'mappings': {
            'properties': {
                'nb_id': {'type': 'integer'},
                'html_url': {'type': 'keyword'},
                'name': {'type': 'keyword'},
                'language': {'type': 'keyword'},
                'markdown': {'type': 'text', 
                             "analyzer":"text_analyzer"
                            },
                'comments': {'type': 'text', 
                             "analyzer":"text_analyzer"
                            },
        }
    }
}
es.indices.create(index = INDEX, body = init_index)

{'acknowledged': True, 'index': 'notebookindex', 'shards_acknowledged': True}

In [22]:
# check if the data and structure is in elasticsearch
# es.search(body={"query": {"match_all": {}}}, index = INDEX)
es.indices.get_mapping(index = INDEX)

{'notebookindex': {'mappings': {'properties': {'comments': {'type': 'text'},
    'html_url': {'type': 'keyword'},
    'language': {'type': 'keyword'},
    'markdown': {'type': 'text'},
    'name': {'type': 'keyword'},
    'nb_id': {'type': 'integer'}}}}}